In [51]:
import os
import requests
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [52]:
!playwright install

In [53]:
SEASONS = ["2024"]

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [54]:
async def get_html(url, selector, sleep=3.01, retries=3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)

        try:
            async with async_playwright() as p:
                # launch the browser instance with playwright
                browser = await p.firefox.launch()

                # create a new tab within the browser
                page = await browser.new_page()

                # navigate to the particular url using the new tab given 
                await page.goto(url)

                print(await page.title())

                html = await page.inner_html(selector)

        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
            
        else:
            break
    
    return html

In [55]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l['href'] for l in links] 
    standings_pages = [f"https://basketball-reference.com{l}" for l in hrefs]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [56]:
season = "2024"
await scrape_season(season)

2023-24 NBA Schedule | Basketball-Reference.com


In [61]:
standings_files = os.listdir(STANDINGS_DIR)
standings_files = [s for s in standings_files if "html" in s and "2024" in s]
standings_files.sort(reverse=True)

standings_files

['NBA_2024_games-october.html',
 'NBA_2024_games-november.html',
 'NBA_2024_games-march.html',
 'NBA_2024_games-january.html',
 'NBA_2024_games-february.html',
 'NBA_2024_games-december.html',
 'NBA_2024_games-april.html']

In [62]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [64]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)

Suns vs Warriors, October 24, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202310250ORL.html
Rockets vs Magic, October 25, 2023 | Basketball-Reference.com
Celtics vs Knicks, October 25, 2023 | Basketball-Reference.com
Wizards vs Pacers, October 25, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202310250CHO.html
Timeout error on https://www.basketball-reference.com/boxscores/202310250CHO.html
Timeout error on https://www.basketball-reference.com/boxscores/202310250CHO.html
Timeout error on https://www.basketball-reference.com/boxscores/202310250MIA.html
Pistons vs Heat, October 25, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202310250TOR.html
Timberwolves vs Raptors, October 25, 2023 | Basketball-Reference.com
Cavaliers vs Nets, October 25, 2023 | Basketball-Reference.com
Pelicans vs Grizzlies, October 25, 2023 | Basketball-Reference.co